In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2019
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
response

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'success': True,
 'result': {'include_total': True,
  'limit': 1000,
  'q': '2019',
  'records_format': 'objects',
  'resource_id': 'd73ee828-c2c6-485c-91dc-c26bd9ce3991',
  'total_estimation_threshold': None,
  'records': [{'_id': 1662,
    'OBJECTID': '1662',
    'Year': '2019',
    'State': 'CA',
    'Agency': 'USF',
    'Unit ID': 'HTF',
    'Fire Name': 'TAMARACK',
    'Local Incident Number': '00030272',
    'Alarm Date': None,
    'Containment Date': None,
    'Cause': '14',
    'Collection Method': '1',
    'Management Objective': '1',
    'GIS Calculated Acres': '67.21748',
    'Comments': 'chanley_nifc;from EGP REST, in July',
    'Complex Name': None,
    'IRWIN ID': None,
    'Fire Number (historical use)': None,
    'Complex ID': None,
    'DECADES': '2010',
    'Shape__Area': '444603.4921875',
    'Shape__Length': '3070.19085492208',
    'rank': 0.057308756},
   {'_id': 1497,
    'OBJECTID': '14

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2019 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2019.columns if col not in columns_to_keep]
df_2019 = df_2019.drop(columns=columns_to_drop)
df_2019 = df_2019.dropna (subset = ["Containment Date"])
df_2019.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
1,2019,11/15/2019 12:00:00 AM,11/15/2019 12:00:00 AM,14,14.53588
2,2019,5/29/0219 12:00:00 AM,5/29/0219 12:00:00 AM,16,2.29694
3,2019,7/31/2019 12:00:00 AM,7/31/2019 12:00:00 AM,14,3.76396
4,2019,10/31/2019 12:00:00 AM,11/5/2019 12:00:00 AM,14,10042.46
5,2019,10/30/2019 12:00:00 AM,11/13/2019 12:00:00 AM,14,1806.234


In [4]:
df_2019['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2019['Alarm Date']]
df_2019['Containment Date'] = [str(item) for item in df_2019['Containment Date']]
df_2019['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2019['Containment Date']]
df_2019.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
1,2019,11/15/2019,11/15/2019,14,14.53588
2,2019,5/29/0219,5/29/0219,16,2.29694
3,2019,7/31/2019,7/31/2019,14,3.76396
4,2019,10/31/2019,11/5/2019,14,10042.46
5,2019,10/30/2019,11/13/2019,14,1806.234


In [5]:
df_2019 = df_2019.rename(columns={'Cause': 'Cause #'})
df_2019['Cause'] = ''
df_2019['Cause #'] = df_2019['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2019['Cause'] = df_2019['Cause #'].apply(map_cause_to_meaning)
df_2019.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
1,2019,11/15/2019,11/15/2019,14,14.53588,Unknown / Unidentified
2,2019,5/29/0219,5/29/0219,16,2.29694,Aircraft
3,2019,7/31/2019,7/31/2019,14,3.76396,Unknown / Unidentified
4,2019,10/31/2019,11/5/2019,14,10042.46,Unknown / Unidentified
5,2019,10/30/2019,11/13/2019,14,1806.234,Unknown / Unidentified


In [6]:
df_2019.to_csv("outputs/fires_2019.csv",
                  encoding="utf-8", index=False, header=True)

In [12]:
# Attempt to convert 'Alarm Date' to datetime, coercing errors
df_2019['Alarm Date'] = pd.to_datetime(df_2019['Alarm Date'], errors='coerce')

# Display rows with invalid dates
invalid_dates = df_2019[df_2019['Alarm Date'].isna()]
print(invalid_dates)

# Drop rows where 'Alarm Date' is in 2017
df_2019 = df_2019[df_2019['Alarm Date'].dt.year != 2017]

# Continue with the processing
df_byalarm = df_2019.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

   Year Alarm Date Containment Date  Cause # GIS Calculated Acres     Cause
2  2019        NaT        5/29/0219       16              2.29694  Aircraft


,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,216,2019,2019-03-13,3/16/2019,14,30.28639,Unknown / Unidentified
1,262,2019,2019-03-14,4/1/2019,1,22.74746,Lightning
2,132,2019,2019-03-15,3/20/2019,7,564.2154,Arson
3,191,2019,2019-04-18,4/18/2019,10,1.908545,Vehicle
4,79,2019,2019-04-20,4/20/2019,9,44.81123,Miscellaneous


In [13]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2019-03-13 00:00:00')

In [15]:
# Convert 'Containment Date' to datetime, coercing errors
df_2019['Containment Date'] = pd.to_datetime(df_2019['Containment Date'], errors='coerce')

# Drop rows with invalid 'Containment Date'
df_2019_filtered = df_2019.dropna(subset=['Containment Date'])

# Now you can sort and process the DataFrame as needed
df_bycontainment = df_2019_filtered.sort_values(by='Containment Date', ascending=False).reset_index(drop=True)
latest_containment = df_bycontainment['Containment Date'].iloc[0]

df_bycontainment.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2019,2019-10-27,2020-01-28,9,703.3557,Miscellaneous
1,2019,2019-07-27,2020-01-27,9,110.7408,Miscellaneous
2,2019,2019-09-04,2020-01-15,9,54614.9,Miscellaneous
3,2019,2019-12-19,2019-12-19,5,12.63458,Debris
4,2019,2019-07-26,2019-12-18,1,6614.634,Lightning


In [16]:
latest_containment

Timestamp('2020-01-28 00:00:00')

In [17]:
output_path = os.path.join("outputs", "timeline_dates_2019.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])